<a href="https://colab.research.google.com/github/TitanLeonov/img/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%962_%D0%9F%D1%80%D0%B8%D0%BC%D0%B5%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%B0_%C2%AB%D1%81%D0%BA%D0%BE%D0%BB%D1%8C%D0%B7%D1%8F%D1%89%D0%B5%D0%B3%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическая работа №2. Применение метода «скользящего окна» для поиска объектов на изображении по эталонам в среде Python. Нелинейные фильтры**

### **Задание №1. Найти пять зашумленных изображений и применить к ним вышеописанные фильтры. Сравнить полученные результаты. Сделать вывод для шумов какого типа целесообразней применение того или иного фильтра**

In [3]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [8]:

image_path = ["im10.png", "im11.png", "im13.jpeg", "im7.png", "im9.png"]

for img in image_path:

  image = cv2.imread(img)

  if image is None:
      print("Ошибка: изображение не найдено по указанному пути.")
  else:
      median_filter = cv2.medianBlur(image, 3)
      sredniy_filter = np.ones((3, 3), np.float32) / 9
      processed_image = cv2.filter2D(image, -1, sredniy_filter)

      combined = np.concatenate((image,median_filter,processed_image ), axis=1)
      cv2_imshow(combined)


Медианный фильтр - подходит для удаления импульсных шумов. Он заменяет каждый пиксель на медиану значений соседних пикселей, сохраняет края объектов, не размывая их.

Средний фильтр -применяется для сглаживания изображения, уменьшая шум. Он заменяет каждый пиксель на среднее значение соседних пикселей, это приводит к размытию краев и общему смягчению изображения.


### **Задание №2. Из подкаталога «Image_1» (каталога IMAGE), определить местонахождение всех фрагментов (Et_1.png - Et_10.png) на растре IMAGE.jpg, используя вышеописанный алгоритм. Вывести результаты в консоль по каждому эталону**


In [9]:
image = cv2.imread('IMAGE.jpg')

if image is None:
    print("Ошибка: не удалось загрузить исходное изображение.")
else:
    etalon_files = ['Et_1.png','Et_2.png', 'Et_3.png', 'Et_4.png', 'Et_5.png', 'Et_6.png', 'Et_7.png', 'Et_8.png', 'Et_9.png', 'Et_10.png']

    for etalon_file in etalon_files:
        fragment = cv2.imread(etalon_file)

        if fragment is None:
            print(f"Ошибка: не удалось загрузить эталон {etalon_file}.")
            continue

        (fragmentHeight, fragmentWidth) = fragment.shape[:2]

        result = cv2.matchTemplate(image, fragment, cv2.TM_CCOEFF)
        (_, _, minLoc, maxLoc) = cv2.minMaxLoc(result)

        topLeft = maxLoc
        botRight = (topLeft[0] + fragmentWidth, topLeft[1] + fragmentHeight)
        roi = image[topLeft[1]:botRight[1], topLeft[0]:botRight[0]].copy()

        mask = np.zeros(image.shape, dtype="uint8")
        image_dark = cv2.addWeighted(image, 0.2, mask, 0.75, 0)

        image_dark[topLeft[1]:botRight[1], topLeft[0]:botRight[0]] = roi

        max_height = 950
        if image_dark.shape[0] > max_height:
            scale = max_height / image_dark.shape[0]
            image_display = cv2.resize(image_dark, None, fx=scale, fy=scale)
        else:
            image_display = image_dark

        print(f"Исходное изображение с выделенным эталоном {etalon_file}:")
        cv2_imshow(image_display)

        print(f"Эталон изображения ({etalon_file}):")
        cv2_imshow(fragment)


### **Задание №3. Создать свой эталон (фрагмент), а затем определить его местонахождение на растре (по образцу из примера выше).**

**Главное при создании фрагмента учесть его разрешение.**

Можно создать эталон на сайте: https://convert-my-image.com/CropImage_Ru или используя срезы NumPy.

In [10]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

image = cv2.imread('Image.jpg')

fragment = cv2.imread('ET(3).jpg')

if image is None:
    print("Ошибка: не удалось загрузить исходное изображение.")
if fragment is None:
    print("Ошибка: не удалось загрузить фрамент изображения.")

(fragmentHeight, fragmentWidth) = fragment.shape[:2]

result = cv2.matchTemplate(image, fragment, cv2.TM_CCOEFF)
(_, _, minLoc, maxLoc) = cv2.minMaxLoc(result)

topLeft = maxLoc
botRight = (topLeft[0] + fragmentWidth, topLeft[1] + fragmentHeight)
roi = image[topLeft[1]:botRight[1], topLeft[0]:botRight[0]].copy()

mask = np.zeros(image.shape, dtype="uint8")
image_dark = cv2.addWeighted(image, 0.2, mask, 0.75, 0)

image_dark[topLeft[1]:botRight[1], topLeft[0]:botRight[0]] = roi

max_height = 950
if image_dark.shape[0] > max_height:
    scale = max_height / image_dark.shape[0]
    image_display = cv2.resize(image_dark, None, fx=scale, fy=scale)
else:
    image_display = image_dark

print("Исходное изображение с выделенным фраментом:")
cv2_imshow(image_display)

print("Фрамент изображения (эталон):")
cv2_imshow(fragment)
